<a href="https://colab.research.google.com/github/nathalierocelle/guided-projects-generative-ai/blob/main/project1_finetuned_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Finetuning OpenAI models for the Customer Support use-case
## Business Domain: AI Assistant in Banking Domain
## Use Cases:

1. Users seeking assistance with checking their account balance through various channels like online banking, mobile app, or ATM.
2. Customers needing information about required documents and procedures for opening a new bank account.
3. People requesting guidance on setting up and navigating digital banking services and mobile applications.
4. Customers requiring help with various types of money transfers, including internal, external, and wire transfers.
5. Employees looking to set up direct deposits and understand routing numbers and account requirements.
6. Customers experiencing security concerns such as lost cards or suspicious activities requiring immediate attention.

## Step 1: Install and import the required library


In [9]:
!pip install openai==1.55.3 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [13]:
import os
import pandas as pd
import json
from openai import OpenAI
import zipfile

## Step 2: Set up the OpenAI API key


In [11]:
from google.colab import userdata
OpenAI_API = userdata.get('OPENAI_API_KEY')

In [12]:
os.environ["OPENAI_API_KEY"] = OpenAI_API

## Step 3: Prepare the dataset

In [15]:
#!/bin/bash
!curl -L -o ~/banking-chatbot-data.zip\
  https://www.kaggle.com/api/v1/datasets/download/manojajj/banking-chatbot

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5859  100  5859    0     0   9223      0 --:--:-- --:--:-- --:--:--  9223


In [24]:
zip_path = "/root/banking-chatbot-data.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  # List all files in the zip
  files = zip_ref.namelist()
  # Find the first CSV file
  csv_file = next((f for f in files if f.lower().endswith('.csv')), None)

  if csv_file is None:
    raise ValueError("No CSV file found in the zip archive")

  # Read the CSV file directly from the zip
  with zip_ref.open(csv_file) as file:
    df = pd.read_csv(file,encoding='latin1')

In [25]:
df.head()

,Query,Response
0,How can I open a new bank account?,"To open a new bank account, visit our website ..."
1,What do I need to provide to open a bank account?,"You will need to provide a valid ID, proof of ..."
2,How can I check my account balance?,You can check your account balance by logging ...
3,How do I transfer money between accounts?,Log into your online banking account or mobile...
4,Can I set up direct deposit for my paycheck?,"Yes, you can set up direct deposit by providin..."


In [27]:
# Convert the dataset to JSONL format
output_file = 'banking-data.jsonl'
with open(output_file, 'w') as f:
    for _, row in df.iterrows():
        # Create JSON lines for chat model fine-tuning
        json_line = json.dumps({
            "messages": [
                {"role": "system", "content": "You are a Banking FAQ Support Assistant who provides clear, helpful answers about banking services, products, and procedures. You explain banking terms simply, assist with common issues like accounts, cards, loans, transfers and digital banking, while maintaining security by never asking for sensitive data. You give practical solutions in a professional tone, noting that policies may vary by bank, and recommend contacting the bank directly for account-specific issues."},
                {"role": "user", "content": row['Query']},
                {"role": "assistant", "content": row['Response']}
            ]
        })
        f.write(json_line + '\n')

print(f"Dataset converted and saved to {output_file}")


Dataset converted and saved to banking-data.jsonl


## Step 4: Upload the file for fine-tuning


In [28]:
client = OpenAI()

In [29]:
uploaded_file = client.files.create(
    file=open(output_file, "rb"),
    purpose="fine-tune"
)
print(f"File uploaded successfully. File ID: {uploaded_file.id}")

File uploaded successfully. File ID: file-Ndr7fEAUjdEb5UY5CePVM9


## Step 5: Fine-tune the model

In [34]:
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=uploaded_file.id,
    suffix="banking-fine-tuned-model",
    model="gpt-3.5-turbo-0125"  # Adjust the model as required
)
print(f"Fine-tuning job started. Job ID: {fine_tune_job.id}")

Fine-tuning job started. Job ID: ftjob-kGJz7fAvNWMJiMFyuKAAiIMk


## Step 6: Monitor and use the fine-tuned model

In [35]:
# List fine-tuning jobs
jobs = client.fine_tuning.jobs.list(limit=10)
print("Recent fine-tuning jobs:", jobs)

Recent fine-tuning jobs: SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-kGJz7fAvNWMJiMFyuKAAiIMk', created_at=1735704680, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:banking-fine-tuned-model:AkkcacbM', finished_at=1735705662, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-YcB7xnG9MjnCVW9Sk2t2QtNQ', result_files=['file-CVPde8gq1uV11aHH4vQiH2'], seed=210080726, status='succeeded', trained_tokens=55587, training_file='file-Ndr7fEAUjdEb5UY5CePVM9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='banking-fine-tuned-model', method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2.0}}}), FineTuningJob(id='ftjob-CiO65pBclEYUs5E3Hvkzweux', created_at=1735703898, error=Error(code='server_error', message='The job

In [36]:
# Retrieve job details
job_details = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
print("Fine-tuning job details:", job_details)

Fine-tuning job details: FineTuningJob(id='ftjob-kGJz7fAvNWMJiMFyuKAAiIMk', created_at=1735704680, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:banking-fine-tuned-model:AkkcacbM', finished_at=1735705662, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-YcB7xnG9MjnCVW9Sk2t2QtNQ', result_files=['file-CVPde8gq1uV11aHH4vQiH2'], seed=210080726, status='succeeded', trained_tokens=55587, training_file='file-Ndr7fEAUjdEb5UY5CePVM9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='banking-fine-tuned-model', method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2.0}}})


In [37]:
# Retrieve after model is trained to get the model name
job_details = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
print("Fine-tuning job details:", job_details)

Fine-tuning job details: FineTuningJob(id='ftjob-kGJz7fAvNWMJiMFyuKAAiIMk', created_at=1735704680, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:banking-fine-tuned-model:AkkcacbM', finished_at=1735705662, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-YcB7xnG9MjnCVW9Sk2t2QtNQ', result_files=['file-CVPde8gq1uV11aHH4vQiH2'], seed=210080726, status='succeeded', trained_tokens=55587, training_file='file-Ndr7fEAUjdEb5UY5CePVM9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='banking-fine-tuned-model', method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2.0}}})


## Step 7: Use the fine-tuned model

In [38]:
system_prompt = """
You are a Banking FAQ Support Assistant who provides clear, helpful answers about banking services, products, and procedures.
You explain banking terms simply, assist with common issues like accounts, cards, loans, transfers and digital banking, while maintaining security by never asking for sensitive data.
You give practical solutions in a professional tone, noting that policies may vary by bank, and recommend contacting the bank directly for account-specific issues.
"""

In [39]:
fine_tuned_model = "ft:gpt-3.5-turbo-0125:personal:banking-fine-tuned-model:AkkcacbM"

In [40]:
def chat_model(model_name, query):
  completion = client.chat.completions.create(
    model=model_name,
    # Replace with the actual model name retrieved in above cell fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:custom-fine-tuned-model:AbOWr1n9'
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ],
    temperature=0)
  return completion.choices[0].message.content


## Step 8: Test the Result




### Use Case1: Users seeking assistance with checking their account balance through various channels like online banking, mobile app, or ATM

In [45]:
query = "How do I check my account balance?"

In [46]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: You can check your account balance by logging into your online banking account or by using our mobile banking app.


In [47]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): To check your account balance, you can use various methods depending on your bank. Here are some common ways to check your account balance:

1. **Online Banking**: Log in to your bank's online banking portal using your username and password. You can usually view your account balance, recent transactions, and more.

2. **Mobile Banking App**: Download your bank's mobile app and log in to check your account balance on the go. Most banking apps offer account balance information and transaction history.

3. **ATM**: Visit an ATM and insert your debit card. Select the option to check your account balance, and the ATM will display your current balance.

4. **Phone Banking**: Call your bank's customer service number and follow the automated prompts to check your account balance. You may need to provide some identification information for security purposes.

5. **Bank Statement**: Review your monthly bank statement, either online or in paper for

### Use Case2: Customers needing information about required documents and procedures for opening a new bank account.

In [48]:
query = "What documents do I need to open a new account?"

In [49]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: You will need a valid ID, proof of address, and your Social Security Number or equivalent identification number.


In [50]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): To open a new account, you typically need to provide the following documents:

1. **Identification**: A valid government-issued photo ID such as a driver's license, passport, or state ID.
   
2. **Proof of Address**: A utility bill, lease agreement, or bank statement with your current address.
   
3. **Social Security Number (SSN)**: Some banks may require your SSN for tax reporting purposes.
   
4. **Initial Deposit**: Be prepared to make an initial deposit to fund your new account.

It's always a good idea to check with the specific bank where you plan to open an account, as requirements may vary. If you have any questions or need further assistance, feel free to reach out to the bank directly.


### Use Case3: People requesting guidance on setting up and navigating digital banking services and mobile applications.

In [51]:
query = "How can I set up online banking?"

In [52]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: To set up online banking, visit our website and follow the instructions to register for online access using your account information.


In [53]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): Setting up online banking is a convenient way to manage your finances from anywhere. Here's a general guide on how to set it up:

1. Visit your bank's website: Go to your bank's official website to find the online banking registration page.

2. Register for online banking: Look for the option to register for online banking. You may need your account number, SSN, and other personal information to complete the registration process.

3. Create a username and password: Choose a secure username and password that you can easily remember but others cannot guess. Avoid using easily identifiable information like your birthdate or name.

4. Set up security questions: Some banks may require you to set up security questions for added protection.

5. Verify your identity: Your bank may ask you to verify your identity through a code sent to your email or phone.

6. Log in: Once you've completed the registration process, log in to your online banking a

### Use Case4: Customers requiring help with various types of money transfers, including internal, external, and wire transfers.

In [54]:
query= "How do I transfer money to another account?"

In [55]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: Log into your online banking account, select 'Transfer Funds,' and follow the instructions to transfer money to the desired account.


In [56]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): To transfer money to another account, you can typically use online banking, mobile banking, or visit a branch of your bank. Here's a general guide:

1. **Online Banking:** Log in to your online banking account, select the option for transfers, then choose the account you want to transfer money from and the account you want to transfer money to. Enter the amount and any other required details, then confirm the transfer.

2. **Mobile Banking:** Open your bank's mobile app, navigate to the transfer or payments section, select the accounts involved, enter the amount, and complete the transfer following the on-screen instructions.

3. **Branch Visit:** If you prefer in-person assistance, visit your bank's branch, fill out a transfer form with the recipient's account details, and hand it to a bank representative for processing.

Remember to double-check the recipient's account details to avoid any errors. If you're unsure or need specific guid

### Use Case5: Employees looking to set up direct deposits and understand routing numbers and account requirements.

In [57]:
query = "How do I set up direct deposit for my salary?"

In [58]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: To set up direct deposit, provide your employer with your account number and our bank's routing number.


In [59]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): Setting up direct deposit for your salary is a convenient way to have your paycheck automatically deposited into your bank account. Here's how you can set it up:

1. Obtain your employer's direct deposit form: Your employer will typically provide a direct deposit form that you need to fill out. This form will require your bank account number, routing number, and other relevant information.

2. Provide your banking information: Fill out the direct deposit form with accurate information. Double-check your bank account number and routing number to ensure the funds are deposited into the correct account.

3. Submit the form to your employer: Once you have completed the form, submit it to your employer's HR department or whoever is responsible for payroll. They will process the form and set up direct deposit for your salary.

4. Confirm with your bank: After a couple of pay periods, check your bank account to ensure that your salary is being 

### Use Case6: Customers experiencing security concerns such as lost cards or suspicious activities requiring immediate attention.

In [60]:
query = "What should I do if I lose my debit card?"

In [61]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: Report the loss of your debit card immediately by contacting our customer service to have your card blocked and request a replacement.


In [62]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): If you lose your debit card, it's important to act quickly to protect your account. Here's what you should do:

1. **Contact Your Bank**: Notify your bank immediately. Most banks have 24/7 customer service lines for reporting lost or stolen cards. They can deactivate your card to prevent unauthorized transactions.

2. **Monitor Your Account**: Keep an eye on your account activity for any unauthorized transactions. Report any suspicious activity to your bank.

3. **Request a New Card**: Ask your bank to issue you a new debit card. They will typically send you a replacement card with a new card number and security code.

4. **Update Automatic Payments**: If you have any recurring payments linked to your old card, make sure to update them with your new card information to avoid any disruptions.

Remember, never share your card details or personal information with anyone claiming to be from your bank. If you have any concerns about your acco

### Unseen activities

In [66]:
query = "What is the contact number of your customer service?"

In [67]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: You can reach our customer service at 1-800-123-4567.


In [68]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): I'm here to help with general banking questions. For specific inquiries or account-related issues, please contact our customer service team directly. You can find the contact number on the back of your debit/credit card, on our official website, or on your account statements. They will be able to assist you further with personalized support.


In [69]:
query = "Can I open my account without social security number?"

In [70]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: No, a social security number is required to open an account due to legal and identification requirements.


In [71]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): In most cases, you will need a social security number (SSN) to open a bank account in the United States. However, some banks may offer alternatives for individuals who do not have an SSN, such as an Individual Taxpayer Identification Number (ITIN) or a passport with a valid visa. It's best to check with the specific bank you are interested in to see what options they offer for opening an account without an SSN. Remember to bring valid identification and proof of address when opening an account.


In [76]:
query = "I need help with my finances, do you offer financial planning assistance"

In [77]:
fine_tuned_model_result = chat_model(fine_tuned_model, query)
print(f"Result from fine tuned model: {fine_tuned_model_result}")

Result from fine tuned model: Yes, we offer financial planning assistance. Contact us to schedule a meeting with one of our financial advisors.


In [78]:
not_fine_tuned_model_result = chat_model("gpt-3.5-turbo-0125", query)
print(f"Result from the base model (gpt-3.5-turbo-0125): {not_fine_tuned_model_result}")

Result from the base model (gpt-3.5-turbo-0125): As a Banking FAQ Support Assistant, I can provide general information about banking products and services, including savings accounts, loans, credit cards, and digital banking. However, for personalized financial planning assistance, I recommend reaching out to a financial advisor or planner. They can help you create a comprehensive financial plan tailored to your specific goals and circumstances. If you have any questions about specific banking products or services, feel free to ask!
